In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualization_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os
import random

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix, Sphere, Rgba
import pydrake.symbolic as sym
import time
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import HPolyhedron, Hyperellipsoid
from pydrake.geometry.optimization_dev import CspaceFreePath
from pydrake.polynomial import Polynomial as PolynomialCommon

In [4]:
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions
import logging
drake_logger = logging.getLogger("drake")
# drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)




pendulum_world = FindResourceOrThrow("drake/C_Iris_Examples/assets/double_pendulum_world.urdf")

models = []
models.append(parser.AddModelFromFile(pendulum_world))

plant.Finalize()
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(plant.num_positions())



# The object we will use to perform our certification.
t0 = time.time()
cspace_free_path = CspaceFreePath(plant, scene_graph, q_star, maximum_path_degree = 1, plane_order = 1)
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity if you want
# to see the plant with the collision geometries.
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, cspace_free_path, viz_role=Role.kIllustration)
visualizer.visualize_collision_constraint(factor = 1.2, num_points = 100)
visualizer.meshcat_cspace.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))
visualizer.meshcat_task_space.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,1,0])))

INFO:drake:Meshcat listening for connections at http://localhost:7000
INFO:drake:Meshcat listening for connections at http://localhost:7001


Time to construct line certifier = 0.004862546920776367s


In [6]:
# compute limits in t-space
limits_q = [plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()]
do_viz = True
# compute limits in s-space
limits_s = []
for q in [plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()]:
    limits_s.append(Ratfk.ComputeSValue(np.array(q), q_star))
limits_s = np.array(limits_s)

In [7]:
# visualizer.meshcat_cspace.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))
# visualizer.meshcat_task_space.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,1,0])))

In [8]:
# # filter fused joints self collisions so they don't interfere with collision engine
# digaram = visualizer.diagram
# context = visualizer.diagram_context
# sg_context = scene_graph.GetMyContextFromRoot(context)
# inspector = scene_graph.model_inspector()

# pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

# gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
# get_name_of_gid = lambda gid : inspector.GetName(gid)
# gids.sort(key=get_name_of_gid)

# id_pairs_of_interest = [(gid1, gid2) for i, gid1 in enumerate(gids) for gid2 in gids[i+1:] ]
# print(id_pairs_of_interest)
# visualizer.collision_pairs_of_interest = id_pairs_of_interest


## Set up the sliders so we can move the plant around manually

In [9]:
sliders = []

for i, q in enumerate(plant.GetPositionLowerLimits()):
    sliders.append(widgets.FloatSlider(min=q,
                                       max=plant.GetPositionUpperLimits()[i], 
                                       value=0, description=f'q{i}'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = np.array([0,0])
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.show_res_q(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


In [10]:
s0 = np.array([0,0])
s1_safe = np.array([-0.25, 0.9])
s1_unsafe = np.array([1,1])

def plot_edge(s0, s1, s0_name = "s0", s1_name = "s1"):
    visualizer.meshcat_cspace.SetObject(f"/{s0_name}",
                                 Sphere(0.05),
                                 Rgba(0.0, 1.0, 0, 1))
    s0_3d = np.array([s0[0],s0[1],0])
    s1_3d = np.array([s1[0],s1[1],0])
    print(s0_3d)
    print(s1_3d)
    visualizer.meshcat_cspace.SetTransform(f"/{s0_name}",
                                  RigidTransform(RotationMatrix(),
                                                 s0_3d))
    
    visualizer.meshcat_cspace.SetObject(f"/{s1_name}",
                                 Sphere(0.05),
                                 Rgba(1.0, 0.0, 0, 1))
    visualizer.meshcat_cspace.SetTransform(f"/{s1_name}",
                                   RigidTransform(RotationMatrix(),
                                                 s1_3d))
    visualizer.meshcat_cspace.SetLine(f"/edge_{s0_name}_{s1_name}", 
                                np.hstack([s0_3d[:,np.newaxis], s1_3d[:, np.newaxis]]),
                                line_width = 2, rgba = Rgba(0.0, 0.0, 0.0, 1))
plot_edge(s0,s1_safe, s1_name = "s1_safe")
plot_edge(s0,s1_unsafe, s1_name = "s1_unsafe")
plot_edge(limits_s[0],limits_s[1], s0_name = "joint limit low", s1_name = "joint limit high")

[0 0 0]
[-0.25  0.9   0.  ]
[0 0 0]
[1 1 0]
[-0.99920399 -0.99920399  0.        ]
[0.99920399 0.99920399 0.        ]


In [11]:
def make_line_poly(start_point, end_point):
    polys = np.empty(shape=(plant.num_positions(), 1), dtype = object)
    for j in range(plant.num_positions()):
        polys[j] = PolynomialCommon(np.array([start_point[j]-end_point[j], end_point[j]]))
    return polys
safe_line = make_line_poly(s0, s1_safe)
unsafe_line = make_line_poly(s0, s1_unsafe)

In [12]:
cert_options = CspaceFreePath.FindSeparationCertificateGivenPathOptions()
cert_options.terminate_segment_certification_at_failure = False

cert_options.num_threads = -1
cert_options.verbose = False
cert_options.solver_id = MosekSolver.id()
cert_options.solver_options = SolverOptions()
cert_options.terminate_path_certification_at_failure = False


In [13]:

is_safe_safe, safe_cert = cspace_free_path.FindSeparationCertificateGivenPath(safe_line, set(), cert_options)
print(is_safe_safe)
is_safe_unsafe, unsafe_cert = cspace_free_path.FindSeparationCertificateGivenPath(unsafe_line, set(), cert_options)
print(is_safe_unsafe)


[True]
[False]


In [14]:
pair, plane_idx = next(iter(cspace_free_path.map_geometries_to_separating_planes().items()))
pair

(<GeometryId value=28>, <GeometryId value=42>)

In [15]:
sep_cert_prog = cspace_free_path.MakeIsGeometrySeparableOnPathProgram(pair, safe_line)

In [16]:
for pair, plane_idx in cspace_free_path.map_geometries_to_separating_planes().items():
    print(pair)

(<GeometryId value=28>, <GeometryId value=42>)
(<GeometryId value=25>, <GeometryId value=42>)


In [17]:
# draw prm
import prm
from pydrake.all import Rgba

# limits = [np.array(t_low), np.array(q_high)]

visualizer.meshcat_cspace.Delete("/prm")

def plot_prm(nodes, adjacency_list, width, color = Rgba(0.0, 0.0, 1, 1), prefix = ""):
    plt_idx = 0
    for node_idx in range(nodes.shape[0]):
        pos1 = np.append(nodes[node_idx, :],0)
        for edge_idx in range(len(adjacency_list[node_idx])): 
            pos2 = np.append(nodes[adjacency_list[node_idx][edge_idx], :],0)
            name = f"/{prefix}/prm/rm/line/{plt_idx}"
#             for c in map(int, sqtring):
#                 name += f"/{c}"
            visualizer.meshcat_cspace.SetLine(name,  np.hstack([pos1[:,np.newaxis], pos2[:, np.newaxis]]),
                                 line_width = width, rgba = color)
            plt_idx +=1
            
plotting_fn_handle_good = partial(plot_prm, width = 0.1, prefix = "Good", color = Rgba(0, 0.,0,1))

def collision(pos, col_func_handle):
    return col_func_handle(pos)


prm_col_fn_handle = partial(collision, col_func_handle = visualizer.check_collision_s_by_ik)

visualizer.check_collision_s_by_ik(np.zeros_like(q_star))
prm_col_fn_handle(np.zeros_like(q_star))


0.0

In [34]:
num_points = 1000

PRM = prm.PRM( 
            limits_s,
            num_points = num_points,
            col_func_handle = prm_col_fn_handle,
            num_neighbours = 5, 
            dist_thresh = .5,
            num_col_checks = 10,
            verbose = True,
            plotcallback = plotting_fn_handle_good
            )
# if num_points < 100:
PRM.plot()
tot_num_edges = len(PRM.adjacency_list)* len(PRM.adjacency_list[0])

[PRM] Samples 0
[PRM] Samples 30
[PRM] Samples 60
[PRM] Samples 90
[PRM] Samples 120
[PRM] Samples 150
[PRM] Samples 180
[PRM] Samples 210
[PRM] Samples 240
[PRM] Samples 270
[PRM] Samples 300
[PRM] Samples 330
[PRM] Samples 360
[PRM] Samples 390
[PRM] Samples 420
[PRM] Samples 450
[PRM] Samples 480
[PRM] Samples 510
[PRM] Samples 540
[PRM] Samples 570
[PRM] Samples 600
[PRM] Samples 630
[PRM] Samples 660
[PRM] Samples 690
[PRM] Samples 720
[PRM] Samples 750
[PRM] Samples 780
[PRM] Samples 810
[PRM] Samples 840
[PRM] Samples 870
[PRM] Samples 900
[PRM] Samples 930
[PRM] Samples 960
[PRM] Samples 990
[PRM] Nodes connected: 0
[PRM] Nodes connected: 20
[PRM] Nodes connected: 40
[PRM] Nodes connected: 60
[PRM] Nodes connected: 80
[PRM] Nodes connected: 100
[PRM] Nodes connected: 120
[PRM] Nodes connected: 140
[PRM] Nodes connected: 160
[PRM] Nodes connected: 180
[PRM] Nodes connected: 200
[PRM] Nodes connected: 220
[PRM] Nodes connected: 240
[PRM] Nodes connected: 260
[PRM] Nodes connected

In [35]:
def make_line_polys(PRM):
    endpoint_index_set = set()
    for neighbors in PRM.adjacency_list:
        for n in neighbors[1:]:
            endpoint_index_set.add((neighbors[0], n))
    polys = np.empty(shape=(plant.num_positions(), len(endpoint_index_set)), dtype = object)
    for i, (idx0, idx1) in enumerate(endpoint_index_set):
        s0 = PRM.nodes[idx0]
        s1 = PRM.nodes[idx1]
        for j in range(plant.num_positions()):
            polys[j,i] = PolynomialCommon(np.array([s0[j]-s1[j], s1[j]]))
    return polys

path_safe = make_line_polys(PRM)

In [36]:
t0 = time.time()
edge_safe, certs = cspace_free_path.FindSeparationCertificateGivenPath(path_safe, set(), cert_options)
t1 = time.time()
print(f"Certification of safe PRM in {t1-t0}s")

Certification of safe PRM in 73.06126642227173s


In [37]:
edge_safe

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [38]:
creation_times = np.zeros(path_safe.shape[1])
cert_times = np.zeros(path_safe.shape[1])
t0_all = time.time()
for i, piece in enumerate(path_safe.T):
    t0 = time.time()
    sep_cert_prog = cspace_free_path.MakeIsGeometrySeparableOnPathProgram(pair, piece)
    t1 = time.time()
    creation_times[i] = t1 -t0
    t0 = time.time()
    res = cspace_free_path.SolveSeparationCertificateProgram(sep_cert_prog, cert_options)
    t1 = time.time()
    cert_times[i] = t1 - t0
    print(f"creation time = {creation_times[i]}")
    print(f"cert time = {cert_times[i]}")
    print()
t1_all = time.time()
print(f"Total Time = {t1_all - t0_all}")

creation time = 0.002701997756958008
cert time = 0.030086278915405273

creation time = 0.0017838478088378906
cert time = 0.0392148494720459

creation time = 0.0020036697387695312
cert time = 0.03077411651611328

creation time = 0.002808094024658203
cert time = 0.0463719367980957

creation time = 0.0028715133666992188
cert time = 0.03772473335266113

creation time = 0.002798318862915039
cert time = 0.028461456298828125

creation time = 0.0028638839721679688
cert time = 0.019249439239501953

creation time = 0.002711057662963867
cert time = 0.0393986701965332

creation time = 0.005848407745361328
cert time = 0.04268789291381836

creation time = 0.002589702606201172
cert time = 0.03545856475830078

creation time = 0.0019598007202148438
cert time = 0.03392815589904785

creation time = 0.0019867420196533203
cert time = 0.03778529167175293

creation time = 0.011379241943359375
cert time = 0.037323951721191406

creation time = 0.00269317626953125
cert time = 0.03582310676574707

creation time 

creation time = 0.002754688262939453
cert time = 0.05059075355529785

creation time = 0.002841472625732422
cert time = 0.03811049461364746

creation time = 0.0028657913208007812
cert time = 0.04534602165222168

creation time = 0.0022706985473632812
cert time = 0.019681692123413086

creation time = 0.002626657485961914
cert time = 0.017912864685058594

creation time = 0.0027778148651123047
cert time = 0.03153681755065918

creation time = 0.0018804073333740234
cert time = 0.045841217041015625

creation time = 0.0027403831481933594
cert time = 0.04509544372558594

creation time = 0.0030651092529296875
cert time = 0.045654296875

creation time = 0.002602815628051758
cert time = 0.041056156158447266

creation time = 0.002722024917602539
cert time = 0.04574847221374512

creation time = 0.002414226531982422
cert time = 0.030478477478027344

creation time = 0.0026149749755859375
cert time = 0.054763078689575195

creation time = 0.0026845932006835938
cert time = 0.05708670616149902

creation ti

creation time = 0.0019497871398925781
cert time = 0.043706655502319336

creation time = 0.002845287322998047
cert time = 0.03804922103881836

creation time = 0.002674579620361328
cert time = 0.06987595558166504

creation time = 0.002207517623901367
cert time = 0.052210092544555664

creation time = 0.001975536346435547
cert time = 0.04187750816345215

creation time = 0.002431631088256836
cert time = 0.03003668785095215

creation time = 0.002619504928588867
cert time = 0.04332900047302246

creation time = 0.002866983413696289
cert time = 0.04719853401184082

creation time = 0.0028085708618164062
cert time = 0.04181408882141113

creation time = 0.002472400665283203
cert time = 0.019054651260375977

creation time = 0.0026743412017822266
cert time = 0.04260134696960449

creation time = 0.002791881561279297
cert time = 0.033881425857543945

creation time = 0.0028717517852783203
cert time = 0.020599365234375

creation time = 0.0024466514587402344
cert time = 0.05158495903015137

creation time

creation time = 0.0026891231536865234
cert time = 0.04121565818786621

creation time = 0.002598285675048828
cert time = 0.03159451484680176

creation time = 0.0025610923767089844
cert time = 0.029155731201171875

creation time = 0.002687692642211914
cert time = 0.06433558464050293

creation time = 0.0027234554290771484
cert time = 0.017258405685424805

creation time = 0.0029134750366210938
cert time = 0.04116940498352051

creation time = 0.002714872360229492
cert time = 0.03895902633666992

creation time = 0.002805948257446289
cert time = 0.03450441360473633

creation time = 0.0023512840270996094
cert time = 0.03295278549194336

creation time = 0.0027081966400146484
cert time = 0.03448796272277832

creation time = 0.002952098846435547
cert time = 0.046849966049194336

creation time = 0.00275421142578125
cert time = 0.04035019874572754

creation time = 0.004937648773193359
cert time = 0.05001473426818848

creation time = 0.0027017593383789062
cert time = 0.03399467468261719

creation ti

creation time = 0.0039136409759521484
cert time = 0.08192253112792969

creation time = 0.0031523704528808594
cert time = 0.0411839485168457

creation time = 0.0024416446685791016
cert time = 0.03493642807006836

creation time = 0.005236148834228516
cert time = 0.06588625907897949

creation time = 0.0016603469848632812
cert time = 0.046343326568603516

creation time = 0.0026297569274902344
cert time = 0.0679013729095459

creation time = 0.0027713775634765625
cert time = 0.017964839935302734

creation time = 0.0024411678314208984
cert time = 0.03390669822692871

creation time = 0.0016856193542480469
cert time = 0.018317222595214844

creation time = 0.002478361129760742
cert time = 0.028984546661376953

creation time = 0.002427339553833008
cert time = 0.035630226135253906

creation time = 0.00238800048828125
cert time = 0.02809429168701172

creation time = 0.0027663707733154297
cert time = 0.030515670776367188

creation time = 0.0027518272399902344
cert time = 0.028512239456176758

creati

...

creation time = 0.0010194778442382812
cert time = 0.007879495620727539

creation time = 0.001020193099975586
cert time = 0.0076487064361572266

creation time = 0.0010132789611816406
cert time = 0.0075685977935791016

creation time = 0.001035451889038086
cert time = 0.007601022720336914

creation time = 0.001013040542602539
cert time = 0.00761723518371582

creation time = 0.0010077953338623047
cert time = 0.011994361877441406

creation time = 0.0011281967163085938
cert time = 0.007681369781494141

creation time = 0.0010766983032226562
cert time = 0.007771015167236328

creation time = 0.0010716915130615234
cert time = 0.011521577835083008

creation time = 0.001100778579711914
cert time = 0.00772857666015625

creation time = 0.0010564327239990234
cert time = 0.007634639739990234

creation time = 0.001043081283569336
cert time = 0.007924079895019531

creation time = 0.0010309219360351562
cert time = 0.007818222045898438

creation time = 0.0010514259338378906
cert time = 0.00764441490173339

creation time = 0.0013003349304199219
cert time = 0.018968582153320312

creation time = 0.0012485980987548828
cert time = 0.030543804168701172

creation time = 0.0011968612670898438
cert time = 0.020208120346069336

creation time = 0.0011687278747558594
cert time = 0.018708467483520508

creation time = 0.001367807388305664
cert time = 0.01851034164428711

creation time = 0.0011570453643798828
cert time = 0.029220104217529297

creation time = 0.0012350082397460938
cert time = 0.018768310546875

creation time = 0.0012841224670410156
cert time = 0.018556594848632812

creation time = 0.0012302398681640625
cert time = 0.018723249435424805

creation time = 0.0012395381927490234
cert time = 0.011933565139770508

creation time = 0.0011417865753173828
cert time = 0.019394397735595703

creation time = 0.0011477470397949219
cert time = 0.01856541633605957

creation time = 0.0011532306671142578
cert time = 0.029221296310424805

creation time = 0.001209259033203125
cert time = 0.018959522247314453


creation time = 0.0011639595031738281
cert time = 0.017569541931152344

creation time = 0.0012290477752685547
cert time = 0.022539615631103516

creation time = 0.0013206005096435547
cert time = 0.019933700561523438

creation time = 0.00115966796875
cert time = 0.019370317459106445

creation time = 0.001171112060546875
cert time = 0.01828765869140625

creation time = 0.0012035369873046875
cert time = 0.021454811096191406

creation time = 0.0012252330780029297
cert time = 0.020461320877075195

creation time = 0.0012469291687011719
cert time = 0.014404058456420898

creation time = 0.0012538433074951172
cert time = 0.01868605613708496

creation time = 0.0011622905731201172
cert time = 0.01905989646911621

creation time = 0.0011982917785644531
cert time = 0.014508962631225586

creation time = 0.0011456012725830078
cert time = 0.021355390548706055

creation time = 0.0012073516845703125
cert time = 0.012610435485839844

creation time = 0.0011260509490966797
cert time = 0.00818634033203125

cr

creation time = 0.001180887222290039
cert time = 0.008808135986328125

creation time = 0.0011277198791503906
cert time = 0.008224248886108398

creation time = 0.0011284351348876953
cert time = 0.012454748153686523

creation time = 0.0012042522430419922
cert time = 0.008384466171264648

creation time = 0.0010800361633300781
cert time = 0.009334802627563477

creation time = 0.00119781494140625
cert time = 0.008276224136352539

creation time = 0.0010418891906738281
cert time = 0.008253335952758789

creation time = 0.0010900497436523438
cert time = 0.008363485336303711

creation time = 0.0011446475982666016
cert time = 0.008376598358154297

creation time = 0.0010399818420410156
cert time = 0.008018732070922852

creation time = 0.0010647773742675781
cert time = 0.008060216903686523

creation time = 0.0010225772857666016
cert time = 0.00806283950805664

creation time = 0.0009729862213134766
cert time = 0.007982730865478516

creation time = 0.0010254383087158203
cert time = 0.0080242156982421

creation time = 0.0012829303741455078
cert time = 0.008618831634521484

creation time = 0.0011174678802490234
cert time = 0.008465290069580078

creation time = 0.0011577606201171875
cert time = 0.008086204528808594

creation time = 0.001094818115234375
cert time = 0.008109569549560547

creation time = 0.0010383129119873047
cert time = 0.007773160934448242

creation time = 0.0010063648223876953
cert time = 0.007908105850219727

creation time = 0.0010342597961425781
cert time = 0.007958173751831055

creation time = 0.0011365413665771484
cert time = 0.008125543594360352

creation time = 0.001004934310913086
cert time = 0.00806117057800293

creation time = 0.000993967056274414
cert time = 0.007779836654663086

creation time = 0.0010645389556884766
cert time = 0.008074045181274414

creation time = 0.0010504722595214844
cert time = 0.007995367050170898

creation time = 0.0010149478912353516
cert time = 0.008201837539672852

creation time = 0.0009877681732177734
cert time = 0.0079672336578369